In [2]:
import spacy
from collections import Counter
import tqdm
import pandas as pd
import pickle

First step is to load in all the files that were created before

In [3]:
file = open("../stored_data/docs.obj",'rb')
cleaned_articles = pickle.load(file)
file.close()

In [4]:
file = open("../stored_data/nouns.obj",'rb')
nouns_per_article = pickle.load(file)
file.close()

In [5]:
file = open("../stored_data/entities.obj",'rb')
entities_manual = pickle.load(file)
file.close()

In [6]:
file = open("../stored_data/NER_train_data.obj",'rb')
TRAIN_DATA = pickle.load(file)
file.close()

Making a prelimanary count of all the proper nouns I can find. I couldn't track a working spacy method so I converted to string and used native Counter.
Also made 2 lists, one containing all unique values and 1 containing all values, both as strings.

In [5]:
all_entities = []
counted_entities = Counter()
unique_entities = set()

for key, val in nouns_per_article.items():
    counted_entities += Counter(val)
    all_entities.append(val)

all_entities = [item for sublist in all_entities for item in sublist]
unique_entities = set(all_entities)

# Save the ones with a minimum of 3 links

counted_entities_thresh = Counter({k: c for k, c in counted_entities.items() if c >= 2})

we have the articles and the pronouns, I think now we want to find the easiest way to classify them.
If I can loop up the entity types, I can only look for the ones that don't have an entity...
Even better if I can loop them up in an existing knowledge base.

In [11]:
import spacy

nlp = spacy.load('en_core_web_lg')

def entity_lookup(doc, uniq):
    recognized_strings = [ent.text for ent in doc.ents]
    matches = set(recognized_strings).intersection(uniq)
    for found in [x for x in matches]:
        uniq.remove(found)
    return uniq

def entity_filter():
    to_be_found = unique_entities
    for key in tqdm.tqdm(list(cleaned_articles.keys())):
        text = ""

        for sent in cleaned_articles[key]:
            text += " " + sent

        doc = nlp(text)

        to_be_found = entity_lookup(doc, to_be_found)
        not_found = to_be_found.copy()

    return not_found

not_found = entity_filter()

100%|██████████| 8399/8399 [04:45<00:00, 29.42it/s]


Now it's time to clean the actual tags

In [7]:
tickers = pd.read_csv('../external_data/bats_symbols.csv')
symbols = tickers.Name.to_list()

In [8]:
def entity_cleaner(entity):
    name, ent = entity
    for sym in symbols:
        name = name.replace('lt'+ sym, " ")

    name = name.lower().replace('  ', ' ')

    return [name, ent]

In [9]:
entities_cleaned = []

for x in entities_manual:
    entities_cleaned.append(entity_cleaner(x))

Save the file

In [ ]:
filehandler = open("../stored_data/entities.obj","wb")
pickle.dump(entities_manual, filehandler)
filehandler.close()

Now the data should be formatted as the actual training data that spacy expects

In [ ]:
TRAIN_DATA = [
   ("Uber blew through $1 million", {"entities": [(0, 4, "ORG")]}),
   ("Google rebrands its apps", {"entities": [(0, 6, "ORG")]})
]

We have to loop over all the articles and find the occurances of each of the entities with the first and last index.

In [8]:
all_articles = []
for id, val in cleaned_articles.items():
    for sent in cleaned_articles[id]:
        all_articles.append(sent)

In [12]:
TRAIN_DATA = []

for ent in tqdm.tqdm(entities_manual):
    for sentence in all_articles:
        if ent[0] in sentence:
                start_index = sentence.find(ent[0])
                end_index = sentence.find(ent[0]) + len(ent[0])
                TRAIN_DATA.append((sentence.lower(), {"entities": [(start_index, end_index, ent[1])]}))
                


  0%|          | 15/3098 [00:00<00:41, 74.52it/s]

Swiss Federal Government 24
swiss federal government
Swiss Federal Government 24
swiss federal government
DIW 3
diw
DIW 3
diw
DIW 3
diw
DIW 3
diw
DIW 3
diw
DIW 3
diw
DIW 3
diw
DIW 3
diw
DIW 3
diw
DIW 3
diw
DIW 3
diw
DIW 3
diw
DIW 3
diw
DIW 3
diw
DIW 3
diw
DIW 3
diw
DIW 3
diw
PARIS PACT 10
paris pact
PARIS PACT 10
paris pact
PARIS PACT 10
paris pact
PARIS PACT 10
paris pact
BAKER FORECASTS 15
baker forecasts
Merchants 9
merchants
Merchants 9
merchants
Merchants 9
merchants
Merchants 9
merchants
Merchants 9
merchants
Merchants 9
merchants
Merchants 9
merchants
Merchants 9
merchants
Merchants 9
merchants
Merchants 9
merchants
Merchants 9
merchants
Merchants 9
merchants
Merchants 9
merchants
Merchants 9
merchants
Merchants 9
merchants
Merchants 9
merchants
Guillaume 9
guillaume
Guillaume 9
guillaume
Guillaume 9
guillaume
Guillaume 9
guillaume
Guillaume 9
guillaume
Guillaume 9
guillaume
Guillaume 9
guillaume
Guillaume 9
guillaume
Guillaume 9
guillaume
US EUROPE JAPAN 15
us europe japan
US E

  1%|          | 23/3098 [00:00<00:43, 71.20it/s]

Koichi Tsukihara Deputy General Manager 39
koichi tsukihara deputy general manager
Koichi Tsukihara Deputy General Manager 39
koichi tsukihara deputy general manager
DLR US 6
dlr us
DLR US 6
dlr us
DLR US 6
dlr us
BELGIUM LAUNCHES 16
belgium launches
BELGIUM LAUNCHES 16
belgium launches
European Court 14
european court
European Court 14
european court
European Court 14
european court
European Court 14
european court
Opecna 6
opecna
Opecna 6
opecna
JP STEVENS 10
jp stevens
JP STEVENS 10
jp stevens
ltDavis Mining 14
ltdavis mining
ltDavis Mining 14
ltdavis mining
CARGILL GRAIN PORT ALLEN 24
cargill grain port allen
CARGILL GRAIN PORT ALLEN 24
cargill grain port allen
CARGILL GRAIN PORT ALLEN 24
cargill grain port allen
Midland Bank Plc ltMDBLL 24
midland bank plc ltmdbll
Midland Bank Plc ltMDBLL 24
midland bank plc ltmdbll
Midland Bank Plc ltMDBLL 24
midland bank plc ltmdbll
Midland Bank Plc ltMDBLL 24
midland bank plc ltmdbll
USASEAN Memorandum 18
usasean memorandum
USASEAN Memorandum 1

  1%|          | 36/3098 [00:01<02:40, 19.03it/s]

...

 11%|█         | 331/3098 [00:08<00:36, 76.49it/s]

ICCO Executive Director Kobena Erbynn 37
icco executive director kobena erbynn
ICCO Executive Director Kobena Erbynn 37
icco executive director kobena erbynn
ICCO Executive Director Kobena Erbynn 37
icco executive director kobena erbynn
ICCO Executive Director Kobena Erbynn 37
icco executive director kobena erbynn
american Stock Exchange 23
american stock exchange
american Stock Exchange 23
american stock exchange
international Monetary Fund IMF 31
international monetary fund imf
international Monetary Fund IMF 31
international monetary fund imf
dlr US 6
dlr us
dlr US 6
dlr us
dlr US 6
dlr us
dlr US 6
dlr us
dlr US 6
dlr us
dlr US 6
dlr us
dlr US 6
dlr us
dlr US 6
dlr us
dlr US 6
dlr us
dlr US 6
dlr us
dlr US 6
dlr us
dlr US 6
dlr us
dlr US 6
dlr us
dlr US 6
dlr us
dlr US 6
dlr us
dlr US 6
dlr us
dlr US 6
dlr us
dlr US 6
dlr us
dlr US 6
dlr us
dlr US 6
dlr us
dlr US 6
dlr us
First Bank Lewiston 19
first bank lewiston
First Bank Lewiston 19
first bank lewiston
BERLINER BANK 13
berliner 

 11%|█         | 332/3098 [00:14<02:00, 22.96it/s]

1
t
t 1
t
t 1
t
t 1
t
t 1
t
t 1
t
t 1
t
t 1
t
t 1
t
t 1
t
t 1
t
t 1
t
t 1
t
t 1
t
t 1
t
t 1
t
t 1
t
t 1
t
t 1
t
t 1
t
t 1
t
t 1
t
t 1
t
t 1
t
t 1
t
t 1
t
t 1
t
t 1
t
t 1
t
t 1
t
t 1
t
t 1
t
t 1
t
t 1
t
t 1
t
t 1
t
t 1
t
t 1
t
t 1
t
t 1
t
t 1
t
t 1
t
t 1
t
t 1
t
t 1
t
t 1
t
t 1
t
t 1
t
t 1
t
t 1
t
t 1
t
t 1
t
t 1
t
t 1
t
t 1
t
t 1
t
t 1
t
t 1
t
t 1
t
t 1
t
t 1
t
t 1
t
t 1
t
t 1
t
t 1
t
t 1
t
t 1
t
t 1
t
t 1
t
t 1
t
t 1
t
t 1
t
t 1
t
t 1
t
t 1
t
t 1
t
t 1
t
t 1
t
t 1
t
t 1
t
t 1
t
t 1
t
t 1
t
t 1
t
t 1
t
t 1
t
t 1
t
t 1
t
t 1
t
t 1
t
t 1
t
t 1
t
t 1
t
t 1
t
t 1
t
t 1
t
t 1
t
t 1
t
t 1
t
t 1
t
t 1
t
t 1
t
t 1
t
t 1
t
t 1
t
t 1
t
t 1
t
t 1
t
t 1
t
t 1
t
t 1
t
t 1
t
t 1
t
t 1
t
t 1
t
t 1
t
t 1
t
t 1
t
t 1
t
t 1
t
t 1
t
t 1
t
t 1
t
t 1
t
t 1
t
t 1
t
t 1
t
t 1
t
t 1
t
t 1
t
t 1
t
t 1
t
t 1
t
t 1
t
t 1
t
t 1
t
t 1
t
t 1
t
t 1
t
t 1
t
t 1
t
t 1
t
t 1
t
t 1
t
t 1
t
t 1
t
t 1
t
t 1
t
t 1
t
t 1
t
t 1
t
t 1
t
t 1
t
t 1
t
t 1
t
t 1
t
t 1
t
t 1
t
t 1
t
t 1
t
t 1
t
t 1
t

KeyboardInterrupt: 

In [24]:
TRAIN_DATA

[('swiss to launch new series of money market paper the swiss federal government will launch a new series of three month money market certificate total around 150 mln swiss franc the national bank say',
  {'entities': [(53, 77, 'GPE')]}),
 ('swiss money market paper yields 3286 pct the swiss federal government s new series of six month money market certificate raise 1775 mln swiss franc at an issue price of 98401 pct to give an average annual yield of 3286 pct the national bank say',
  {'entities': [(45, 69, 'GPE')]}),
 ('german industrial employment see stagnate the number of worker employ in the west german industrial sector stagnate in the last quarter of 1986 as a 50000 increase in overall employment benefit only the service branch the diw economic institute say',
  {'entities': [(222, 225, 'GPE')]}),
 ('a diw report add the general downturn in the economy since last autumn have have a negative effect on the willingness of firm to take on worker',
  {'entities': [(2, 5, 'GPE')]}),


In [25]:
filehandler = open("../stored_data/NER_train_data.obj","wb")
pickle.dump(TRAIN_DATA, filehandler)
filehandler.close()